# ChatGPT Prefrontal Cortex



Copyright 2023, [Jozsef Szalma](https://www.linkedin.com/in/szalma/)<br>
Creative Commons Attribution-NonCommercial 4.0 International Public License <br>
https://creativecommons.org/licenses/by-nc/4.0/legalcode <br>
If you want to use the below code commercially then how about you contact me first, thanks.<br>

In [1]:
#dependencies

#%pip install openai
#%pip install ipywidgets
#%pip install pyperclip

In [2]:
import os
import openai
import json

from IPython.display import display
from IPython.display import HTML
import ipywidgets as widgets

import pyperclip

### API

In [3]:
#api key
openai.api_key = os.getenv("KEY")

#default personality / behavior for the chatbot (e.g. "You are a helpful and knowledgable assistant"); this is also where you might want to spell out the role of the inhibitor model, so the main model knows what to do with it
default_system = os.getenv("PROMPT_SYSTEM")

#default instructions to the inhibitor model (i.e. what to look for, formatting the feedback in json etc)
#the below code is expecting the responses from the inhibitor in the following format: 
# {"decision": "pass", "explanation": ""}
# {"decision": "inhibit", "explanation": ""}
#so you need to engineer your prompt accordingly
default_inhibitor = os.getenv("PROMPT_INHIBITOR")

#chat model parameters (ChatGPT)
chat_model = "gpt-3.5-turbo"
chat_temp = 0.7
chat_convo_limit = 3000
chat_token_limit = 1000

#inhibitor model parameters (GPT3)
inhibitor_model = "text-davinci-003"
inhibitor_temp = 0.0
inhibitor_token_limit = 1000

#TODO refactor to get rid of global variables

if (default_system is not None):
    #initializing chat history with the system prompt
    conversation = [{"role": "system", "content": default_system}]
else:
    conversation = []
    
#the maximum context window (short term memory) is 4096 tokens, so we need to keep track of token usage to limit the risk running into an API error; the web UI is using a sliding window to manage around this limit
token_count = 0


#API calls to OpenAI
def get_gpt (prompt):
    global token_count
    global conversation

    #if our current token_count is above limit, then we remove the oldest message from the conversation, if there is a personality set up in default_system then the 2nd oldest
    if token_count > chat_convo_limit:
   
        prompt_length = len(prompt)

        while prompt_length > 0 :
   
            if (default_system is not None):
                item_length = len(conversation[1]['content'])
                conversation.pop(1)
            else:
                item_length = len(conversation[0]['content'])
                conversation.pop(0)

            prompt_length = prompt_length - item_length

    conversation.append({"role": "user", "content": prompt})
    
    #sending user prompt to primary model
    response = openai.ChatCompletion.create(
            model=chat_model, 
            messages=conversation,
            temperature = chat_temp,
            max_tokens = chat_token_limit,
            n = 1

        )
    message = response.choices[0].message.content    
    conversation.append({"role": "assistant", "content": message}) 
    
    #sending instructions to inhibitor model + primary model's last response. 
    #it is a design decision if the entire chat history should be sent to the inhibitor (more context) or only the last message (less bias?)

    inhibitor_response = openai.Completion.create(
            model = inhibitor_model,
            prompt = default_inhibitor + " " + message,
            temperature = chat_temp,
            max_tokens = inhibitor_token_limit,
            n = 1
        )
    
    
    inhibitor_message = inhibitor_response.choices[0].text 
    #the inhibitor's responses are added to the conversation for transparency
    conversation.append({"role": "system", "content": inhibitor_message})   
    try:
        inhibitor_decision = json.loads(inhibitor_message)['decision']

        #if the inhibitor has an issue the primary model gets another opportunity to rephrase its response.
    
        if (inhibitor_decision.lower() == 'inhibit'):
            response = openai.ChatCompletion.create(
                model=chat_model, 
                messages=conversation,
                temperature = chat_temp,
                max_tokens = chat_token_limit,
                n = 1

            )
            message = response.choices[0].message.content    
        
            conversation.append({"role": "assistant", "content": message}) 
    except:
        print('inhibitor returned malformed response: ', inhibitor_message)    
    token_count = response.usage.total_tokens
    
    return response

### UI

In [ ]:

#HTML formatting for the output window
html_template = """
<style>
div {{white-space: pre-wrap;font-size: 18px;}}

</style>
<div>
<div>
{}
</div>
<div>
{}
</div>
<div>
{}
</div>
</div>
"""

#displaying messages
def render_message(message):
    if message["role"] == "assistant":
        return (f"<i>ChatGPT</i>: {message['content']}")
    elif message["role"] == "system":
        return (f"<i>System</i>: {message['content']}")
    elif message["role"] == "user":
        return (f"<b>You</b>: {message['content']}")
    
# handler for copy button's on_click event
def handle_copy(button):
    global conversation
    
    if (default_system is not None):
        conversation_str = "\n".join([f"{msg['role']}: {msg['content']}" for msg in conversation[1:]])
    else:
        conversation_str = "\n".join([f"{msg['role']}: {msg['content']}" for msg in conversation])

    # Add conversation string to clipboard
    pyperclip.copy(conversation_str)

# handler for send button's on_click event
def handle_input(button):
    user_input = input_widget.value
    output.append_display_data("--------------message sent--------------")
    input_widget.value = ''
    
    response = get_gpt(user_input)
    
    latest_messages = html_template.format(render_message(conversation[-3]),render_message(conversation[-2]),render_message(conversation[-1]))
    output.append_display_data(HTML(latest_messages))

    print('current token count: ', token_count)


output = widgets.Output(layout={'border': '1px solid black','width': '100%'})

#input window
input_widget = widgets.Textarea(
    value='',
    placeholder='Type your message here...',
    description='Input:',
    disabled=False,
    layout={'width': '100%', 'height': 'auto','font-size':'20px'},
    rows = 1
)

#resizing input window as the input gets longer    
def get_bigger(args):        
    input_widget.rows = input_widget.value.count('\n') + 1

input_widget.observe(get_bigger,'value')

#the send button
send_button = widgets.Button(description='Send', layout={'width': '100%'})
send_button.on_click(handle_input)

#the copy to clipboard button
copy_button = widgets.Button(description='Copy Chat to Clipboard', layout={'width': '100%'})
copy_button.on_click(handle_copy)


# Display the UI
display(widgets.VBox([copy_button,output,input_widget,send_button]))



In [ ]:
conversation